In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [3]:
df = pd.read_csv('548Var_FinalDF.csv')

In [4]:
len(df)

1000000

In [5]:
df = df.set_index('record')

In [6]:
## exclude out last 2 month
mydata = df.iloc[:833507]
# exclude first 2 weeks
mydata = mydata.iloc[38511:]
# Segment the data
OOT = df.iloc[833507:]

In [311]:
top_30 = ['address_zip5_0_count30_count_ratio',
 'address_count3_date',
 'address_zip5_0_count180_count_ratio',
 'address_0_count7_count_ratio',
 'homephone_address_count30_date',
 'address_zip5_0_count3_count_ratio',
 'homephone_address_pastday',
 'homephone_address_zip5_count90_date',
 'ssn_dob_count90_date',
 'address_1_count90_count_ratio',
 'ssn_dob_0_count30_count_ratio',
 'firstname_lastname_dob_0_count30_count_ratio',
 'ssn_0_count30_count_ratio',
 'ssn_firstname_lastname_dob_0_count30_count_ratio',
 'ssn_firstname_0_count30_count_ratio',
 'address_zip5_1_count90_count_ratio',
 'ssn_firstname_count7_date',
 'address_count0_date',
 'homephone_address_0_count180_count_ratio',
 'address_zip5_1_count7_count_ratio',
 'firstname_lastname_dob_0_count7_count_ratio',
 'ssn_dob_1_count30_count_ratio',
 'homephone_zip_count1_date',
 'ssn_firstname_1_count180_count_ratio',
 'homephone_address_zip5_1_count7_count_ratio',
 'homephone_count180_date',
 'address_firstname_lastname_dob_count30_date',
 'zip5_firstname_lastname_dob_count30_date',
 'homephone_firstname_lastname_count30_date',
 'zip5_firstname_lastname_dob_0_count30_count_ratio',
 'fraud_label']

In [312]:
len(top_30)

31

In [313]:
final_df = mydata[top_30]
final_OOT = OOT[top_30]

In [314]:
X = final_df.drop('fraud_label',axis = 1)
y = final_df['fraud_label']

In [315]:
X_train,X_test,y_train,y_test = train_test_split (X,y,test_size = 0.25, random_state = 33)

In [316]:
final_OOT_X = final_OOT.drop('fraud_label',axis = 1)
final_OOT_y = final_OOT['fraud_label']

In [317]:
from sklearn.svm import LinearSVC

In [318]:
top_20 = top_30[0:20]
top_20.append('fraud_label')

In [319]:
final_df1 = mydata[top_20]
final_OOT1 = OOT[top_20]

In [320]:
X1 = final_df1.drop('fraud_label',axis = 1)
y1 = final_df1['fraud_label']

In [321]:
X_train1,X_test1,y_train1,y_test1 = train_test_split (X1,y1,test_size = 0.25, random_state = 33)

In [322]:
final_OOT1_x = final_OOT1.drop('fraud_label',axis = 1)
final_OOT1_y = final_OOT1['fraud_label']

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
from sklearn.ensemble import RandomForestClassifier

# Logistic Model

In [323]:
lr = LogisticRegression(C= 1, penalty='l1',solver='liblinear',random_state=None)
lr.fit(X_train,y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [324]:
np.sum(lr.coef_ == 0)

11

In [329]:
lm1 = list(pd.DataFrame(lr.predict_proba(X_train))[1])
lm2 = list(pd.DataFrame(lr.predict_proba(X_test))[1])
lm3 = list(pd.DataFrame(lr.predict_proba(final_OOT_X))[1])

# Random Forest Model

In [335]:
clf=RandomForestClassifier(n_estimators=700,min_samples_leaf= 8,max_depth=15, max_features=7)
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features=7,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=700,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [340]:
rf1 = list(pd.DataFrame(clf.predict_proba(X_train))[1])
rf2 = list(pd.DataFrame(clf.predict_proba(X_test))[1])
rf3 = list(pd.DataFrame(clf.predict_proba(final_OOT_X))[1])

# Linear SVC

In [131]:
from sklearn.calibration import CalibratedClassifierCV

In [347]:
lsvc = CalibratedClassifierCV(base_estimator=LinearSVC(penalty='l2',C = 0.01, dual=False))
lsvc.fit(X_train,y_train)

CalibratedClassifierCV(base_estimator=LinearSVC(C=0.01, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                loss='squared_hinge',
                                                max_iter=1000,
                                                multi_class='ovr', penalty='l2',
                                                random_state=None, tol=0.0001,
                                                verbose=0),
                       cv=None, method='sigmoid')

In [348]:
lsvc1 = list(pd.DataFrame(lsvc.predict_proba(X_train))[1])
lsvc2 = list(pd.DataFrame(lsvc.predict_proba(X_test))[1])
lsvc3 = list(pd.DataFrame(lsvc.predict_proba(final_OOT_X))[1])

# Boosted Tree

In [349]:
gbc = GradientBoostingClassifier(learning_rate = 0.1,max_depth =3,n_estimators=1500)
gbc.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=1500,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [354]:
gbc1 = list(pd.DataFrame(gbc.predict_proba(X_train))[1])
gbc2 = list(pd.DataFrame(gbc.predict_proba(X_test))[1])
gbc3 = list(pd.DataFrame(gbc.predict_proba(final_OOT_X))[1])

# NN

In [378]:
NN1 = list(pd.read_csv('NN_train.csv')['0'])
NN2 = list(pd.read_csv('NN_test.csv')['0'])
NN3 = list(pd.read_csv('NN_oot.csv')['0'])

# Ensemble

In [405]:
l1 = list(np.array([rf1,gbc1]).mean(axis = 0))
l2 = list(np.array([rf2,gbc2]).mean(axis = 0))
l3 = list(np.array([rf3,gbc3]).mean(axis = 0))

In [419]:
len(l3)

166493

In [420]:
sum(final_OOT_y)

2386

In [421]:
len(l3) - sum(final_OOT_y)

164107

In [406]:
# train
tr = pd.DataFrame(list(zip(l1,y_train)), columns=['predicted','fraud'])
tr = tr.sort_values(by = ['predicted'],ascending = False)
sum(list(tr[:int(np.floor(0.03*len(tr)))].fraud))/sum(list(y_train))

0.559403054681124

In [407]:
# test
te = pd.DataFrame(list(zip(l2,y_test)), columns=['predicted','fraud'])
te = te.sort_values(by = ['predicted'],ascending = False)
sum(list(te[:int(np.floor(0.03*len(te)))].fraud))/sum(list(y_test))

0.5431419731866621

In [408]:
# OOT
# test
oo = pd.DataFrame(list(zip(l3,final_OOT_y)), columns=['predicted','fraud'])
oo = oo.sort_values(by = ['predicted'],ascending = False)
sum(list(oo[:int(np.floor(0.03*len(oo)))].fraud))/sum(list(final_OOT_y))

0.5146689019279128

In [422]:
# train 
train_final = X_train.copy()
test_final = X_test.copy()
OOT_final = final_OOT1_x.copy()

In [423]:
train_final['fraud_label'] = list(y_train)
test_final['fraud_label'] = list(y_test)
OOT_final['fraud_label'] = list(final_OOT_y)
# not sorted prediction 
train_final['pred'] = gbc1
test_final['pred'] = gbc2
OOT_final['pred'] = gbc3

In [424]:
train_final = train_final.sort_values(by = ['pred'],ascending = False)
test_final = test_final.sort_values(by = ['pred'],ascending = False)
OOT_final = OOT_final.sort_values(by = ['pred'],ascending = False)

In [425]:
count = 0
for dataset in [train_final,test_final,OOT_final]:
    
    start = 0
    bin_num = int(np.floor(len(dataset)*0.01 + 1))
    rec, good, bad, pgood, pbad, crec, cgood, cbad, pcgood, pcbad, ks, fpr = [],[],[],[],[],[],[],[],[],[],[],[]
    
    

    for i in range(1,21):
        # seperate_bin 
        num_bads = sum(dataset.iloc[start:bin_num*i].fraud_label)
        num_goods = len(dataset.iloc[start:bin_num*i].fraud_label) - sum(dataset.iloc[start:bin_num*i].fraud_label)
        per_goods = num_goods/len(dataset.iloc[start:bin_num*i].fraud_label)
        per_bads = num_bads/len(dataset.iloc[start:bin_num*i].fraud_label)

        rec.append(bin_num)
        good.append(num_goods)
        bad.append(num_bads)
        pgood.append(per_goods)
        pbad.append(per_bads)

        start += bin_num


        # cumulative_bin
        cum_records = bin_num*i
        cum_bads = sum(dataset.iloc[:cum_records].fraud_label)
        cum_goods = len(dataset.iloc[:cum_records].fraud_label) - sum(dataset.iloc[:cum_records].fraud_label)
        cum_per_goods = cum_goods/len(dataset[dataset.fraud_label == 0])
        FDR_cum_per_bads = cum_bads/len(dataset[dataset.fraud_label == 1])
        KS = FDR_cum_per_bads - cum_per_goods
        FPR = cum_goods/cum_bads

        crec.append(cum_records)
        cgood.append(cum_goods)
        cbad.append(cum_bads)
        pcgood.append(cum_per_goods)
        pcbad.append(FDR_cum_per_bads)
        ks.append(KS)
        fpr.append(FPR)
    
    temp = pd.DataFrame([rec, good, bad, pgood, pbad,crec, cgood, cbad, pcgood, pcbad, ks, fpr]).transpose()
    temp.to_excel('df' + str(count) + '.xlsx')
    count+=1


# for 30

In [350]:
l1 = list(pd.DataFrame(gbc.predict_proba(X_train))[1])
l2 = list(pd.DataFrame(gbc.predict_proba(X_test))[1])
l3 = list(pd.DataFrame(gbc.predict_proba(final_OOT_X))[1])

In [351]:
# train
tr = pd.DataFrame(list(zip(l1,y_train)), columns=['predicted','fraud'])
tr = tr.sort_values(by = ['predicted'],ascending = False)
sum(list(tr[:int(np.floor(0.03*len(tr)))].fraud))/sum(list(y_train))

0.5597528273289029

In [352]:
# test
te = pd.DataFrame(list(zip(l2,y_test)), columns=['predicted','fraud'])
te = te.sort_values(by = ['predicted'],ascending = False)
sum(list(te[:int(np.floor(0.03*len(te)))].fraud))/sum(list(y_test))

0.5427982124441388

In [353]:
# OOT
# test
oo = pd.DataFrame(list(zip(l3,final_OOT_y)), columns=['predicted','fraud'])
oo = oo.sort_values(by = ['predicted'],ascending = False)
sum(list(oo[:int(np.floor(0.03*len(oo)))].fraud))/sum(list(final_OOT_y))

0.5150880134115675

# for 20

In [252]:
l1 = list(pd.DataFrame(lr.predict_proba(X_train1))[1])
l2 = list(pd.DataFrame(lr.predict_proba(X_test1))[1])
l3 = list(pd.DataFrame(lr.predict_proba(final_OOT1_x))[1])

In [253]:
# train
tr = pd.DataFrame(list(zip(l1,y_train1)), columns=['predicted','fraud'])
tr = tr.sort_values(by = ['predicted'],ascending = False)
sum(list(tr[:int(np.floor(0.03*len(tr)))].fraud))/sum(list(y_train1))

0.5138160195872683

In [254]:
# test
te = pd.DataFrame(list(zip(l2,y_test1)), columns=['predicted','fraud'])
te = te.sort_values(by = ['predicted'],ascending = False)
sum(list(te[:int(np.floor(0.03*len(te)))].fraud))/sum(list(y_test1))

0.5063595737366793

In [255]:
# OOT
oo = pd.DataFrame(list(zip(l3,final_OOT1_y)), columns=['predicted','fraud'])
oo = oo.sort_values(by = ['predicted'],ascending = False)
sum(list(oo[:int(np.floor(0.03*len(oo)))].fraud))/sum(list(final_OOT1_y))

0.490360435875943